# 1. Linear Regression

In [1]:
import numpy as np
import pandas as pd

In [2]:
class CustomLinearRegression:
    def __init__(self, X_data, y_target, learning_rate=0.01, num_epochs=10000):
        self.num_samples = X_data.shape[0]
        self.X_data = np.c_[np.ones((self.num_samples, 1)), X_data]
        self.y_target = y_target
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs

        # Initial weights
        self.theta = np.random.randn(self.X_data.shape[1], 1)
        self.losses = []

    def compute_loss(self, y_pred, y_target):
        loss = np.dot((y_pred - y_target).T, (y_pred - y_target)) / (self.X_data.shape[0])
        return loss

    def predict(self, X_data):
        y_pred = X_data.dot(self.theta)
        return y_pred

    def fit(self):
        for epoch in range(self.num_epochs):
            # Predict
            y_pred = self.predict(self.X_data)

            # Compute loss
            loss = self.compute_loss(y_pred, self.y_target)
            self.losses.append(loss)

            # Compute gradient
            loss_grd = 2 * (y_pred - self.y_target) / self.num_samples
            gradients = self.X_data.T.dot(loss_grd)

            # Update weight
            self.theta = self.theta - self.learning_rate * gradients

            if (epoch % 50) == 0:
                print(f"Epoch: {epoch} - Loss: {loss}")

        return {
            "loss": sum(self.losses) / len(self.losses),
            "weight": self.theta
            }

In [3]:
# Evaluation
def r2score(y_pred, y):
    rss = np.sum((y_pred - y) ** 2)
    tss = np.sum((y - y.mean()) ** 2)
    r2 = 1 - (rss / tss)
    return r2

In [4]:
# Case 1
y_pred = np.array([1, 2, 3, 4, 5])
y = np.array([1, 2, 3, 4, 5])
r2score(y_pred, y)

1.0

In [5]:
# Case 2
y_pred = np.array([1, 2, 3, 4, 5])
y = np.array([3, 5, 5, 2, 4])
r2score(y_pred, y)

-2.235294117647059

# 2. Polynomial Regression

In [6]:
def create_polynomial_features(X, degree=2):
    """Creates the polynomial features
    Args:
        X: A array tensor for the data.
        degree: A intege for the degree of
        the generated polynomial function.
    """
    X_new = X
    for d in range(2, degree + 1):
        X_new = np.c_[X_new, np.power(X, d)]

    return X_new

In [7]:
X = np.array([[1], [2], [3]])
create_polynomial_features(X)

array([[1, 1],
       [2, 4],
       [3, 9]])

In [8]:
def create_polynomial_features(X, degree=2):
    """ Creates the polynomial features
    Args:
        X: A array for the data.
        degree: A intege for the degree of
        the generated polynomial function.
    """
    X_mem = []
    for X_sub in X.T:
        X_new = X_sub
        for d in range(2, degree + 1):
            X_new = np.c_[X_new, np.power(X_sub, d)]
        X_mem.extend(X_new.T)

    return np.c_[X_mem].T

In [9]:
X = np.array([[1, 2],
              [2, 3],
              [3, 4]])
degree = 2
create_polynomial_features(X, degree)

array([[ 1,  1,  2,  4],
       [ 2,  4,  3,  9],
       [ 3,  9,  4, 16]])

# 3. Sales Prediction

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [11]:
# Load dataset
df = pd.read_csv("/content/SalesPrediction.csv")
df.head()

,TV,Radio,Social Media,Influencer,Sales
0,16.0,6.566231,2.907983,Mega,54.732757
1,13.0,9.237765,2.409567,Mega,46.677897
2,41.0,15.886446,2.913410,Mega,150.177829
3,83.0,30.020028,6.922304,Mega,298.246340
4,15.0,8.437408,1.405998,Micro,56.594181


In [12]:
# Preprocessing

# One Hot Encoding
df = pd.get_dummies(df)
df.head()

,TV,Radio,Social Media,Sales,Influencer_Macro,Influencer_Mega,Influencer_Micro,Influencer_Nano
0,16.0,6.566231,2.907983,54.732757,False,True,False,False
1,13.0,9.237765,2.409567,46.677897,False,True,False,False
2,41.0,15.886446,2.913410,150.177829,False,True,False,False
3,83.0,30.020028,6.922304,298.246340,False,True,False,False
4,15.0,8.437408,1.405998,56.594181,False,False,True,False


In [13]:
# Handle Null values
df = df.fillna(df.mean())

# Get features
X = df[["TV", "Radio", "Social Media", "Influencer_Macro", "Influencer_Mega", "Influencer_Micro", "Influencer_Nano"]]
y = df[["Sales"]]

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

In [14]:
# Feature Scaling
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train)
X_test_processed = scaler.transform(X_test)
scaler.mean_[0]

54.173577723283785

In [15]:
# Polynomial Features
poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train_processed)
X_test_poly = poly_features.transform(X_test_processed)

In [16]:
# Training and Evaluation
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)
preds = poly_model.predict(X_test_poly)
r2_score(y_test, preds)

0.9951796158547627